# OpenAI Streaming - Traced With LastMile

Four use cases:

1. Text - Non-streaming
2. [this one] Text - Streaming
3. Tool Calls - Non-streaming
4. Tool Calls - Streaming

## How to generate function arguments

In [1]:
# !pip install scipy --quiet
# !pip install tenacity --quiet
# !pip install tiktoken --quiet
# !pip install termcolor --quiet
# !pip install openai --quiet
# !pip install "tracing-auto-instrumentation[openai]"
!pip list | grep tracing_auto_instrumentation

# Create ~/.env file with this line: OPENAI_API_KEY=<your key here>
# You can get your key from https://platform.openai.com/api-keys 
import openai
import dotenv
import os
dotenv.load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

tracing_auto_instrumentation             0.0.4     /Users/rossdancraig/Projects/tracing_auto_instrumentation


In [2]:
import openai

from lastmile_eval.rag.debugger.api.tracing import LastMileTracer

from tracing_auto_instrumentation.openai import openai as openai_tracing
from lastmile_eval.rag.debugger.tracing.sdk import get_lastmile_tracer

tracer: LastMileTracer = get_lastmile_tracer(
    tracer_name="OpenAI Text Calling w. Streaming",
)
client = openai.OpenAI(api_key=OPENAI_API_KEY)
client = openai_tracing.wrap(client, tracer)

/opt/homebrew/Caskroom/miniconda/base/envs/eval/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-05-29 15:11:55,666 - Starting new HTTPS connection (1): lastmileai.dev:443
2024-05-29 15:11:55,779 - https://lastmileai.dev:443 "GET /api/evaluation_projects/list?name=OpenAI+Text+Calling+w.+Streaming HTTP/1.1" 200 367
2024-05-29 15:11:55,781 - load_ssl_context verify=True cert=None trust_env=True http2=False
2024-05-29 15:11:55,782 - load_verify_locations cafile='/opt/homebrew/Caskroom/miniconda/base/envs/eval/lib/python3.12/site-packages/certifi/cacert.pem'


In [3]:
def run_my_existing_openai_app(user_message: str, stream: bool = True):
    completion_params = {
        "model": "gpt-3.5-turbo",
        "top_p": 1,
        "max_tokens": 3000,
        "temperature": 1,
        "stream": stream,
        "messages": [
            {
                "content": user_message,
                "role": "user",
            }
        ],
    }

    response = client.chat.completions.create(**completion_params)
    print("Chat Completion Response: ")
    if stream:
        for chunk in response:
            print(chunk)
    else:
        print(response)

In [4]:
# Run your code as usual
run_my_existing_openai_app("Tell me a joke about apples")

2024-05-29 15:11:55,809 - Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'content': 'Tell me a joke about apples', 'role': 'user'}], 'model': 'gpt-3.5-turbo', 'max_tokens': 3000, 'stream': True, 'temperature': 1, 'top_p': 1}}
2024-05-29 15:11:55,820 - Sending HTTP Request: POST https://api.openai.com/v1/chat/completions
2024-05-29 15:11:55,821 - connect_tcp.started host='api.openai.com' port=443 local_address=None timeout=5.0 socket_options=None
2024-05-29 15:11:55,861 - connect_tcp.complete return_value=<httpcore._backends.sync.SyncStream object at 0x168e9af60>
2024-05-29 15:11:55,861 - start_tls.started ssl_context=<ssl.SSLContext object at 0x168ea7750> server_hostname='api.openai.com' timeout=5.0
2024-05-29 15:11:55,877 - start_tls.complete return_value=<httpcore._backends.sync.SyncStream object at 0x168d34890>
2024-05-29 15:11:55,877 - send_request_headers.started request=<Request [b'POST']>
2024-05-29 15:11:55,878 - send_

Chat Completion Response: 
fuck this starting create now
starting this stupid dumbass span


2024-05-29 15:11:56,305 - receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Date', b'Wed, 29 May 2024 19:11:56 GMT'), (b'Content-Type', b'text/event-stream; charset=utf-8'), (b'Transfer-Encoding', b'chunked'), (b'Connection', b'keep-alive'), (b'openai-organization', b'lastmile-ai'), (b'openai-processing-ms', b'147'), (b'openai-version', b'2020-10-01'), (b'strict-transport-security', b'max-age=15724800; includeSubDomains'), (b'x-ratelimit-limit-requests', b'10000'), (b'x-ratelimit-limit-tokens', b'2000000'), (b'x-ratelimit-remaining-requests', b'9999'), (b'x-ratelimit-remaining-tokens', b'1996992'), (b'x-ratelimit-reset-requests', b'6ms'), (b'x-ratelimit-reset-tokens', b'90ms'), (b'x-request-id', b'req_f26ddc0caeb01b3e1a741dacb40cfdfb'), (b'CF-Cache-Status', b'DYNAMIC'), (b'Set-Cookie', b'__cf_bm=vKjWHAnFACZVrNAY8agIROhqlKfMaANG5O3nL6oXLVw-1717009916-1.0.1.1-i7yFPceqitnAKe88CVa12wl1SynzSilKiBg35JpUVn_v0pX0p.r7fBTVxpamShE_RHGRwP63FR2to_vUsBAhbg; path=/; expires

ChatCompletionChunk(id='chatcmpl-9UJ0GDFgCH0BpdFPmXicAKxxCA96T', choices=[Choice(delta=ChoiceDelta(content='', function_call=None, role='assistant', tool_calls=None), finish_reason=None, index=0, logprobs=None)], created=1717009916, model='gpt-3.5-turbo-0125', object='chat.completion.chunk', system_fingerprint=None, usage=None)
ChatCompletionChunk(id='chatcmpl-9UJ0GDFgCH0BpdFPmXicAKxxCA96T', choices=[Choice(delta=ChoiceDelta(content='Why', function_call=None, role=None, tool_calls=None), finish_reason=None, index=0, logprobs=None)], created=1717009916, model='gpt-3.5-turbo-0125', object='chat.completion.chunk', system_fingerprint=None, usage=None)
ChatCompletionChunk(id='chatcmpl-9UJ0GDFgCH0BpdFPmXicAKxxCA96T', choices=[Choice(delta=ChoiceDelta(content=' did', function_call=None, role=None, tool_calls=None), finish_reason=None, index=0, logprobs=None)], created=1717009916, model='gpt-3.5-turbo-0125', object='chat.completion.chunk', system_fingerprint=None, usage=None)
ChatCompletionChu

2024-05-29 15:11:56,689 - receive_response_body.complete
2024-05-29 15:11:56,690 - response_closed.started
2024-05-29 15:11:56,690 - response_closed.complete
2024-05-29 15:11:56,695 - Starting new HTTPS connection (1): lastmileai.dev:443
2024-05-29 15:11:56,845 - https://lastmileai.dev:443 "POST /api/trace/create HTTP/1.1" 200 10
2024-05-29 15:11:56,850 - Starting new HTTPS connection (1): lastmileai.dev:443


ChatCompletionChunk(id='chatcmpl-9UJ0GDFgCH0BpdFPmXicAKxxCA96T', choices=[Choice(delta=ChoiceDelta(content=' doctor', function_call=None, role=None, tool_calls=None), finish_reason=None, index=0, logprobs=None)], created=1717009916, model='gpt-3.5-turbo-0125', object='chat.completion.chunk', system_fingerprint=None, usage=None)
ChatCompletionChunk(id='chatcmpl-9UJ0GDFgCH0BpdFPmXicAKxxCA96T', choices=[Choice(delta=ChoiceDelta(content='?', function_call=None, role=None, tool_calls=None), finish_reason=None, index=0, logprobs=None)], created=1717009916, model='gpt-3.5-turbo-0125', object='chat.completion.chunk', system_fingerprint=None, usage=None)
ChatCompletionChunk(id='chatcmpl-9UJ0GDFgCH0BpdFPmXicAKxxCA96T', choices=[Choice(delta=ChoiceDelta(content=' Because', function_call=None, role=None, tool_calls=None), finish_reason=None, index=0, logprobs=None)], created=1717009916, model='gpt-3.5-turbo-0125', object='chat.completion.chunk', system_fingerprint=None, usage=None)
ChatCompletionC

2024-05-29 15:11:56,942 - https://lastmileai.dev:443 "POST /api/rag_query_traces/create HTTP/1.1" 200 475
2024-05-29 15:11:56,944 - Starting new HTTPS connection (1): lastmileai.dev:443
2024-05-29 15:11:57,146 - https://lastmileai.dev:443 "POST /api/rag_events/create HTTP/1.1" 200 808


Setting up the OpenAI Wrapper is as easy as the following 5 lines.

In [ ]:
# OpenAI Setup
from tenacity import retry, stop_after_attempt, wait_random_exponential
from termcolor import colored
import json

import openai

# GPT_MODEL = "gpt-4o"
GPT_MODEL = "gpt-3.5-turbo"

### Utilities

First let's define a few utilities for making calls to the Chat Completions API and for maintaining and keeping track of the conversation state.

In [ ]:
@retry(wait=wait_random_exponential(multiplier=1, max=40), stop=stop_after_attempt(3))
def chat_completion_request(messages, tools=None, tool_choice=None, model=GPT_MODEL, stream=True, stream_options=None):
    try:
        kwargs = {
            "model": model,
            "messages": messages,
            "tools": tools,
            "tool_choice": tool_choice,
            "stream": stream,
            "n": 3,
        }
        if stream_options is not None:
            kwargs["stream_options"] = stream_options
        response = client.chat.completions.create(**kwargs)
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e


In [ ]:
def pretty_print_conversation(messages):
    role_to_color = {
        "system": "red",
        "user": "green",
        "assistant": "blue",
        "function": "magenta",
    }
    
    for message in messages:
        if message["role"] == "system":
            print(colored(f"system: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "user":
            print(colored(f"user: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "assistant" and message.get("function_call"):
            print(colored(f"assistant: {message['function_call']}\n", role_to_color[message["role"]]))
        elif message["role"] == "assistant" and not message.get("function_call"):
            print(colored(f"assistant: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "function":
            print(colored(f"function ({message['name']}): {message['content']}\n", role_to_color[message["role"]]))


### Basic concepts

Let's create some function specifications to interface with a hypothetical weather API. We'll pass these function specification to the Chat Completions API in order to generate function arguments that adhere to the specification.

In [ ]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "format": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                        "description": "The temperature unit to use. Infer this from the users location.",
                    },
                },
                "required": ["location", "format"],
            },
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_n_day_weather_forecast",
            "description": "Get an N-day weather forecast",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "format": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                        "description": "The temperature unit to use. Infer this from the users location.",
                    },
                    "num_days": {
                        "type": "integer",
                        "description": "The number of days to forecast",
                    }
                },
                "required": ["location", "format", "num_days"]
            },
        }
    },
]

If we prompt the model about the current weather, it will respond with some clarifying questions.

In [ ]:
from tracing_auto_instrumentation.openai.openai import postprocess_streaming_results
import json

messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "What's the weather like today"})
stream = False
kwargs = {
    "messages": messages,
    "tools": tools,
    "stream": stream,
}
if stream:
    kwargs["stream_options"] = {"include_usage": True}
chat_response = chat_completion_request(
    **kwargs
)
print("about to process this bullshit")
if stream:
    all_response_chunks = []
    for chunk in chat_response:
        print(chunk.choices)
        all_response_chunks.append(chunk)
    print(f"{all_response_chunks=}")
    with open("wrapper_chunks_with_usage.json", "w") as f:
        dict_chunks = list(map(lambda x: x.model_dump(), all_response_chunks))
        f.write(json.dumps(dict_chunks, indent=2))
    # response = postprocess_streaming_results(all_response_chunks)
    # assistant_message = response["message"]
    # messages.append(assistant_message)
else:
    assistant_message = chat_response.choices[0].message
    messages.append(assistant_message)
    assistant_message


In [ ]:
g = (n for n in range(3, 5))
for i in g:
    print(i)
print(g)

In [ ]:


print(chat_response)
for i in chat_response:
    print(i)
    print("got here")
print("done")

Once we provide the missing information, it will generate the appropriate function arguments for us.

In [ ]:
# messages.append({"role": "user", "content": "I'm in Glasgow, Scotland."})
stream = False
kwargs = {
    "messages": messages,
    "tools": tools,
    "stream": stream,
}
if stream:
    kwargs["stream_options"] = {"include_usage": True}
chat_response = chat_completion_request(
    **kwargs
)
if stream:
    all_response_chunks = []
    for chunk in chat_response:
        print(chunk)
        all_response_chunks.append(chunk)
print(chat_response)
# assistant_message = chat_response.choices[0].message
# messages.append(assistant_message)
# assistant_message


By prompting it differently, we can get it to target the other function we've told it about.

In [ ]:
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "what is the weather going to be like in Glasgow, Scotland over the next x days"})
chat_response = chat_completion_request(
    messages, tools=tools
)
assistant_message = chat_response.choices[0].message
messages.append(assistant_message)
assistant_message


Once again, the model is asking us for clarification because it doesn't have enough information yet. In this case it already knows the location for the forecast, but it needs to know how many days are required in the forecast.

In [ ]:
messages.append({"role": "user", "content": "5 days"})
chat_response = chat_completion_request(
    messages, tools=tools
)
chat_response.choices[0]


#### Forcing the use of specific functions or no function

We can force the model to use a specific function, for example get_n_day_weather_forecast by using the function_call argument. By doing so, we force the model to make assumptions about how to use it.

In [ ]:
# in this cell we force the model to use get_n_day_weather_forecast
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "Give me a weather report for Toronto, Canada."})
chat_response = chat_completion_request(
    messages, tools=tools, tool_choice={"type": "function", "function": {"name": "get_n_day_weather_forecast"}}
)
chat_response.choices[0].message

In [ ]:
# if we don't force the model to use get_n_day_weather_forecast it may not
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "Give me a weather report for Toronto, Canada."})
chat_response = chat_completion_request(
    messages, tools=tools
)
chat_response.choices[0].message

We can also force the model to not use a function at all. By doing so we prevent it from producing a proper function call.

In [ ]:
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "Give me the current weather (use Celcius) for Toronto, Canada."})
chat_response = chat_completion_request(
    messages, tools=tools, tool_choice="none"
)
chat_response.choices[0].message


### Parallel Function Calling

Newer models such as gpt-4o or gpt-3.5-turbo can call multiple functions in one turn.

In [ ]:
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "what is the weather going to be like in San Francisco and Glasgow over the next 4 days"})
chat_response = chat_completion_request(
    messages, tools=tools, model=GPT_MODEL
)

assistant_message = chat_response.choices[0].message.tool_calls
assistant_message

## How to call functions with model generated arguments

In our next example, we'll demonstrate how to execute functions whose inputs are model-generated, and use this to implement an agent that can answer questions for us about a database. For simplicity we'll use the [Chinook sample database](https://www.sqlitetutorial.net/sqlite-sample-database/).

*Note:* SQL generation can be high-risk in a production environment since models are not perfectly reliable at generating correct SQL.

### Specifying a function to execute SQL queries

First let's define some helpful utility functions to extract data from a SQLite database.

In [ ]:
!mkdir data

In [ ]:
import sqlite3

conn = sqlite3.connect("data/Chinook.db")
print("Opened database successfully")

In [ ]:
def get_table_names(conn):
    """Return a list of table names."""
    table_names = []
    tables = conn.execute("SELECT name FROM sqlite_master WHERE type='table';")
    for table in tables.fetchall():
        table_names.append(table[0])
    return table_names


def get_column_names(conn, table_name):
    """Return a list of column names."""
    column_names = []
    columns = conn.execute(f"PRAGMA table_info('{table_name}');").fetchall()
    for col in columns:
        column_names.append(col[1])
    return column_names


def get_database_info(conn):
    """Return a list of dicts containing the table name and columns for each table in the database."""
    table_dicts = []
    for table_name in get_table_names(conn):
        columns_names = get_column_names(conn, table_name)
        table_dicts.append({"table_name": table_name, "column_names": columns_names})
    return table_dicts


Now can use these utility functions to extract a representation of the database schema.

In [ ]:
database_schema_dict = get_database_info(conn)
database_schema_string = "\n".join(
    [
        f"Table: {table['table_name']}\nColumns: {', '.join(table['column_names'])}"
        for table in database_schema_dict
    ]
)

As before, we'll define a function specification for the function we'd like the API to generate arguments for. Notice that we are inserting the database schema into the function specification. This will be important for the model to know about.

In [ ]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "ask_database",
            "description": "Use this function to answer user questions about music. Input should be a fully formed SQL query.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": f"""
                                SQL query extracting info to answer the user's question.
                                SQL should be written using this database schema:
                                {database_schema_string}
                                The query should be returned in plain text, not in JSON.
                                """,
                    }
                },
                "required": ["query"],
            },
        }
    }
]

### Executing SQL queries

Now let's implement the function that will actually excute queries against the database.

In [ ]:
def ask_database(conn, query):
    """Function to query SQLite database with a provided SQL query."""
    try:
        results = str(conn.execute(query).fetchall())
    except Exception as e:
        results = f"query failed with error: {e}"
    return results

##### Steps to invoke a function call using Chat Completions API: 

**Step 1**: Prompt the model with content that may result in model selecting a tool to use. The description of the tools such as a function names and signature is defined in the 'Tools' list and passed to the model in API call. If selected, the function name and parameters are included in the response.<br>
  
**Step 2**: Check programmatically if model wanted to call a function. If true, proceed to step 3. <br>  
**Step 3**: Extract the function name and parameters from response, call the function with parameters. Append the result to messages. <br>    
**Step 4**: Invoke the chat completions API with the message list to get the response. 

In [ ]:
# Step #1: Prompt with content that may result in function call. In this case the model can identify the information requested by the user is potentially available in the database schema passed to the model in Tools description. 
messages = [{
    "role":"user", 
    "content": "What is the name of the album with the most tracks?"
}]

response = client.chat.completions.create(
    model='gpt-4o', 
    messages=messages, 
    tools= tools, 
    tool_choice="auto"
)

# Append the message to messages list
response_message = response.choices[0].message 
messages.append(response_message)

print(response_message)

In [ ]:
# Step 2: determine if the response from the model includes a tool call.   
tool_calls = response_message.tool_calls
if tool_calls:
    # If true the model will return the name of the tool / function to call and the argument(s)  
    tool_call_id = tool_calls[0].id
    tool_function_name = tool_calls[0].function.name
    tool_query_string = eval(tool_calls[0].function.arguments)['query']
    
    # Step 3: Call the function and retrieve results. Append the results to the messages list.      
    if tool_function_name == 'ask_database':
        results = ask_database(conn, tool_query_string)
        
        messages.append({
            "role":"tool", 
            "tool_call_id":tool_call_id, 
            "name": tool_function_name, 
            "content":results
        })
        
        # Step 4: Invoke the chat completions API with the function response appended to the messages list
        # Note that messages with role 'tool' must be a response to a preceding message with 'tool_calls'
        model_response_with_function_call = client.chat.completions.create(
            model="gpt-4o",
            messages=messages,
        )  # get a new response from the model where it can see the function response
        print(model_response_with_function_call.choices[0].message.content)
    else: 
        print(f"Error: function {tool_function_name} does not exist")
else: 
    # Model did not identify a function to call, result can be returned to the user 
    print(response_message.content) 

## Next Steps

See our other [notebook](How_to_call_functions_for_knowledge_retrieval.ipynb) that demonstrates how to use the Chat Completions API and functions for knowledge retrieval to interact conversationally with a knowledge base.